<a href="https://colab.research.google.com/github/md499/uiuc-cs-598-final/blob/master/PyHealth_Contribution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

In [2]:
base_data = pd.read_csv('/content/data.csv')
base_data.head()

,Path,Sex,Age,Frontal/Lateral,AP/PA,Enlarged Cardiomediastinum,Cardiomegaly,Lung Opacity,Lung Lesion,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices,No Finding
0,CheXpert-v1.0/train/patient00001/study1/view1_...,Female,68,Frontal,AP,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,CheXpert-v1.0/train/patient00002/study2/view1_...,Female,87,Frontal,AP,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
2,CheXpert-v1.0/train/patient00002/study1/view1_...,Female,83,Frontal,AP,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0
3,CheXpert-v1.0/train/patient00002/study1/view2_...,Female,83,Lateral,NaN,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0
4,CheXpert-v1.0/train/patient00003/study1/view1_...,Male,41,Frontal,AP,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
## Take a random subset of 10,000 records for the pyhealth mapping

pyhealth_data = base_data.sample(n=10000, random_state=42)
pyhealth_data.head()

,Path,Sex,Age,Frontal/Lateral,AP/PA,Enlarged Cardiomediastinum,Cardiomegaly,Lung Opacity,Lung Lesion,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices,No Finding
305,CheXpert-v1.0/train/patient00082/study1/view1_...,Male,18,Frontal,PA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
151324,CheXpert-v1.0/train/patient35759/study9/view1_...,Male,90,Frontal,AP,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
19284,CheXpert-v1.0/train/patient04789/study1/view2_...,Male,39,Lateral,NaN,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0
165026,CheXpert-v1.0/train/patient38491/study5/view1_...,Male,60,Frontal,AP,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
26786,CheXpert-v1.0/train/patient06537/study3/view1_...,Female,66,Frontal,AP,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [ ]:
### Generate a list of labels and automate the mapping

In [4]:
LABEL_COLUMNS = [
    'Enlarged Cardiomediastinum', 'Cardiomegaly', 'Lung Opacity',
    'Lung Lesion', 'Edema', 'Consolidation'
]


In [5]:
### pyhealth
from pyhealth.datasets import MIMIC3Dataset
mimic3base = MIMIC3Dataset(
    root="https://storage.googleapis.com/pyhealth/Synthetic_MIMIC-III/",
    tables=["DIAGNOSES_ICD", "PROCEDURES_ICD", "PRESCRIPTIONS"],
)

INFO: Pandarallel will run on 4 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
finish basic patient information parsing : 24.577637195587158s
finish parsing DIAGNOSES_ICD : 20.59243631362915s
finish parsing PROCEDURES_ICD : 16.037468671798706s
finish parsing PRESCRIPTIONS : 59.651694536209106s


Mapping codes: 100%|██████████| 49993/49993 [00:00<00:00, 59855.86it/s]


In [6]:
mimic3base.tables

['DIAGNOSES_ICD', 'PROCEDURES_ICD', 'PRESCRIPTIONS']

In [7]:
import pandas as pd

# 1. Load CheXpert label file
chexpert_df = pd.read_csv('/content/data.csv')

# 2. Sample label to ICD-10 mapping (extend as needed)
label_to_icd = {
    'Cardiomegaly': 'I51.7',
    'Edema': 'R60.9',
    'Consolidation': 'J98.4',
    'Atelectasis': 'J98.11',
    'Pleural Effusion': 'J90',
    'Pneumonia': 'J18.9',
    'Pneumothorax': 'J93.9',
    'Fracture': 'S22.3',
    'Lung Lesion': 'D14.3',
    'Enlarged Cardiomediastinum': 'I51.7',
    # Add more as needed
}

# 3. Function to convert CheXpert row to ICD codes
def get_icd_codes_from_labels(row):
    icd_codes = []
    for label, icd in label_to_icd.items():
        if pd.notnull(row[label]) and row[label] == 1.0:
            icd_codes.append(icd)
    return icd_codes

# 4. Process dataset
chexpert_df['ICD_Codes'] = chexpert_df.apply(get_icd_codes_from_labels, axis=1)

# 5. Output example structure for PyHealth integration
processed_df = chexpert_df[['Path', 'ICD_Codes']]
processed_df['Path'] = processed_df['Path'].apply(lambda x: '/content/CheXpert-v1.0-small/' + x)

# Save or return the processed dataframe
processed_df.to_csv('chexpert_icd_mapped.csv', index=False)

print(processed_df.head())

<ipython-input-7-0c8707658bc1>:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  processed_df['Path'] = processed_df['Path'].apply(lambda x: '/content/CheXpert-v1.0-small/' + x)


                                                Path  \
0  /content/CheXpert-v1.0-small/CheXpert-v1.0/tra...   
1  /content/CheXpert-v1.0-small/CheXpert-v1.0/tra...   
2  /content/CheXpert-v1.0-small/CheXpert-v1.0/tra...   
3  /content/CheXpert-v1.0-small/CheXpert-v1.0/tra...   
4  /content/CheXpert-v1.0-small/CheXpert-v1.0/tra...   

                                           ICD_Codes  
0                                                 []  
1          [I51.7, R60.9, J98.4, J98.11, J90, I51.7]  
2  [I51.7, J98.4, J98.11, J18.9, S22.3, D14.3, I5...  
3  [I51.7, J98.4, J98.11, J18.9, S22.3, D14.3, I5...  
4                              [I51.7, R60.9, I51.7]  


In [43]:
new_data= pd.read_csv('/content/chexpert_icd_mapped.csv')

new_data.head()

,Path,ICD_Codes
0,/content/CheXpert-v1.0-small/CheXpert-v1.0/tra...,[]
1,/content/CheXpert-v1.0-small/CheXpert-v1.0/tra...,"['I51.7', 'R60.9', 'J98.4', 'J98.11', 'J90', '..."
2,/content/CheXpert-v1.0-small/CheXpert-v1.0/tra...,"['I51.7', 'J98.4', 'J98.11', 'J18.9', 'S22.3',..."
3,/content/CheXpert-v1.0-small/CheXpert-v1.0/tra...,"['I51.7', 'J98.4', 'J98.11', 'J18.9', 'S22.3',..."
4,/content/CheXpert-v1.0-small/CheXpert-v1.0/tra...,"['I51.7', 'R60.9', 'I51.7']"


In [52]:
import pandas as pd
import ast

# Load the CheXpert file
new_data = pd.read_csv('/content/chexpert_icd_mapped.csv')

# Ensure ICD_Codes is a list, not a string
if isinstance(new_data["ICD_Codes"].iloc[0], str):
    new_data["ICD_Codes"] = new_data["ICD_Codes"].apply(ast.literal_eval)

# Load your ICD-10 → ICD-9 mapping file
mapping_df = pd.read_csv('/content/icd10cmtoicd9gem.csv')

# Build the mapping dictionary
icd10_to_icd9_dict = mapping_df.groupby("icd10cm")["icd9cm"].apply(list).to_dict()

# Define the mapping function
def map_icd10_to_icd9(icd10_list):
    icd9_codes = set()
    for code in icd10_list:
        normalized = code.replace(".", "").upper()
        if normalized in icd10_to_icd9_dict:
            icd9_codes.update(icd10_to_icd9_dict[normalized])
    return list(icd9_codes)

# Apply the function to map ICD-10 to ICD-9
new_data["ICD9_Mapped"] = new_data["ICD_Codes"].apply(map_icd10_to_icd9)

# Preview the result
new_data[["Path", "ICD_Codes", "ICD9_Mapped"]].head()

,Path,ICD_Codes,ICD9_Mapped
0,/content/CheXpert-v1.0-small/CheXpert-v1.0/tra...,[],[]
1,/content/CheXpert-v1.0-small/CheXpert-v1.0/tra...,"[I51.7, R60.9, J98.4, J98.11, J90, I51.7]","[7823, 5111, 51889, 4293, 5180, 51189]"
2,/content/CheXpert-v1.0-small/CheXpert-v1.0/tra...,"[I51.7, J98.4, J98.11, J18.9, S22.3, D14.3, I5...","[51889, 4293, 5180, 486]"
3,/content/CheXpert-v1.0-small/CheXpert-v1.0/tra...,"[I51.7, J98.4, J98.11, J18.9, S22.3, D14.3, I5...","[51889, 4293, 5180, 486]"
4,/content/CheXpert-v1.0-small/CheXpert-v1.0/tra...,"[I51.7, R60.9, I51.7]","[4293, 7823]"


In [54]:
new_data.to_csv('final_pyhealth_mapped.csv', index=False)


In [ ]:
### push this dataset to pyhealth branch  ###